In [ ]:
! pip install --upgrade codetiming numpy pandas census us mechanicalsoup geopandas pandas_bokeh torch
get_ipython().kernel.do_shutdown(True)

In [1]:
mount_path = '/content/drive'
repo_path = f'{mount_path}/MyDrive/gerrymandering/2022-10/voting_predictor'
from google.colab import drive
drive.mount(mount_path)
%cd {repo_path}
%load_ext google.colab.data_table
%load_ext autoreload
%autoreload
from acs5 import *

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/gerrymandering/2022-10/voting_predictor
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
